In [23]:
import pandas as pd
import dask.dataframe as dd
from dask.delayed import delayed
from zipfile import ZipFile
import hvplot.pandas
import hvplot.dask

In [2]:
def load_zipped_csv(file_path):
    zipped_file = [delayed(pd.read_csv)(file_path)]
    return dd.from_delayed(zipped_file)

## Load the data

In [3]:
df_order_products__prior = load_zipped_csv("../data/order_products__prior.csv.zip")
df_orders = load_zipped_csv("../data/orders.csv.zip")
df_products = load_zipped_csv("../data/products.csv.zip")
df_aisles = load_zipped_csv("../data/aisles.csv.zip")
df_departments = load_zipped_csv("../data/departments.csv.zip")

In [4]:
df_orders = df_orders[df_orders["eval_set"] == "prior"]

In [5]:
df = df_orders.join(df_order_products__prior, on='order_id', rsuffix="_")
df = df.join(df_products, on='product_id', rsuffix="_")
df = df.join(df_aisles, on='aisle_id', rsuffix="_")
df = df.join(df_departments, on='department_id', rsuffix="_")

df.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,order_id_,product_id,add_to_cart_order,reordered,product_id_,product_name,aisle_id,department_id,aisle_id_,aisle,department_id_,department
0,2539329,1,prior,1,2,8,NaN,267793,23236,6,1,23237.0,Organic Chunky Tomato Sauce,81.0,15.0,82.0,baby accessories,16.0,dairy eggs
1,2398795,1,prior,2,3,7,15.0,253040,39409,7,1,39410.0,Gatherings Turkey & Cheese Snack Tray,1.0,20.0,2.0,specialty cheeses,21.0,missing
2,473747,1,prior,3,3,12,21.0,50089,15400,3,0,15401.0,Dry Sweeping Pad Refills With Febreze Lavender...,100.0,21.0,101.0,air fresheners candles,NaN,NaN
3,2254736,1,prior,4,4,7,29.0,237822,41787,12,0,41788.0,Organic Creamy Cashewmilk,91.0,16.0,92.0,baby food formula,17.0,household
4,431534,1,prior,5,4,15,28.0,45690,38293,22,0,38294.0,"Pizza, Margherita, Rice Crust",79.0,1.0,80.0,deodorants,2.0,other


In [7]:
n_rows = len(df)
n_rows

3214874

## Straight forward data visualization

In [ ]:
plot_df = df.sample(frac=0.1, random_state=0)
len(plot_df)

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 6))
# ax.set_title("When do customer order products?")
# ax = sns.histplot(data=plot_df, x="order_hour_of_day", bins=12, stat="probability")
# plt.show()

plot_df.hvplot.hist("order_hour_of_day", title="When do customer order products?")

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 6))
# ax.set_title("When do customer order products?")
# ax = sns.histplot(data=df, x="order_hour_of_day", bins=12, stat="probability")
# plt.show()

In [26]:
df.groupby("order_hour_of_day").order_id.count().compute().to_frame().reset_index()

,order_hour_of_day,order_id
0,0,21372
1,1,11596
2,2,7070
3,3,5120
4,4,5175
5,5,8972
6,6,28792
7,7,86656
8,8,168321
9,9,243496


In [28]:
# df_plot = df.groupby("order_hour_of_day").order_id.count().compute()/n_rows
import hvplot.pandas
df.groupby("order_hour_of_day").order_id.count().compute().to_frame().reset_index().hvplot.bar(x="order_hour_of_day", y="order_id")

:Bars   [order_hour_of_day]   (order_id)

In [32]:
import hvplot.pandas
%time df.groupby("order_hour_of_day").order_id.count().compute().to_frame().reset_index().hvplot.bar(x="order_hour_of_day", y="order_id")

CPU times: user 10.9 s, sys: 1.37 s, total: 12.3 s
Wall time: 12.3 s


:Bars   [order_hour_of_day]   (order_id)

In [48]:
%time test_df = df.sample(frac=0.1, random_state=0).compute()

CPU times: user 11.1 s, sys: 1.55 s, total: 12.6 s
Wall time: 12.6 s


In [51]:
import hvplot.pandas
%time test_df.groupby("order_hour_of_day").order_id.count().reset_index().hvplot.bar(x="order_hour_of_day", y="order_id")

CPU times: user 26.3 ms, sys: 0 ns, total: 26.3 ms
Wall time: 24.3 ms


:Bars   [order_hour_of_day]   (order_id)